# Fine-tuning LLMs with Gender-inclusive Text

## Set-up and imports

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed May 22 08:30:42 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   49C    P8              13W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
! pip install -q datasets evaluate accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.5 MB/s eta 0:00:00


In [ ]:
! pip uninstall -y -q transformers && pip install -q git+https://github.com/huggingface/transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
! huggingface-cli whoami

Not logged in


In [ ]:
! huggingface-cli login
# login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, AutoModelForMaskedLM, AutoConfig
from peft import LoraConfig, TaskType, get_peft_model
from datasets import load_dataset
import torch
import numpy as np
import evaluate
import os
import time
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('Using {} device'.format(device))

Using cuda device


In [ ]:
manipulation = 'neutral-R' # neutral pronouns + replacements
# manipulation = '-R' # only replacements

In [ ]:
data_path = '/content/drive/MyDrive/Projects/fine-tuning/tiny_heap-{0}.txt'.format(manipulation)

In [ ]:
save_folder = '/content/drive/MyDrive/Projects/fine-tuning/'

## Load Dataset

In [ ]:
if os.path.isfile(data_path):
        print('data_dir is a file')
        data_type = 'file'
        num_samples = sum(1 for line in open(data_path))
        dataset = load_dataset("text", data_files={"train": data_path}, streaming=True)

else:
    print('data_dir is a directory')
    num_samples = 0
    for dirpath, dirnames, filenames in os.walk(data_path):
        for filename in filenames:
            if filename.endswith('.txt'):
                num_samples += sum(1 for line in open(os.path.join(dirpath, filename)))

    # Prepare your training data
    dataset = load_dataset(data_path, streaming=True)

print('num_samples: ', num_samples)

data_dir is a file
num_samples:  11591


## Encode dataset

In [ ]:
batch_size = 2  # 8 = default
#max_len = 1024 # for CausalLM

def encode(examples):
        result = tokenizer(
            examples['text'], truncation=True, padding='max_length', max_length=max_len)
        result['labels'] = result['input_ids'].copy()
        return result

tokenized_dataset = dataset.map(encode, batched=True, batch_size=batch_size)

small_eval_dataset = tokenized_dataset['train'].shuffle(seed=42).take(50)
small_train_dataset = tokenized_dataset['train'].shuffle(seed=42).skip(50)

## Load Model

In [ ]:
model_name = 'gpt2'
# model_name = 'microsoft/phi-1_5'
# model_name = 'FacebookAI/roberta-large'

In [ ]:
if 'bert' in model_name:
    model = AutoModelForMaskedLM.from_pretrained(model_name)
else:
    model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


trainable params: 147,456 || all params: 124,587,264 || trainable%: 0.1184


/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1119: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token

### With Transformers

In [ ]:
epochs = 3 # 3 default
batch_size = 2  # 8 = default
gradient_accumulation_steps = 1  # 1 = default

# from: https://stackoverflow.com/questions/76011298/huggingface-trainer-max-step-to-set-for-streaming-dataset
max_steps = (
    num_samples // batch_size) // gradient_accumulation_steps * epochs
print('max_steps: ', max_steps)

max_steps:  17385


In [ ]:
if model_name == 'gpt2' or 'phi' in model_name:
    max_len = tokenizer.model_max_length
elif 'llama' in model_name:
    config = AutoConfig.from_pretrained(model_name)
    max_len = config.hidden_size
elif 'bert' in model_name:
    max_len = 512
print('max_len:', max_len)

max_len: 1024


In [ ]:
perplexity_metric = evaluate.load('perplexity', module_type='metric')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return perplexity_metric.compute(predictions=predictions, model_id=model_name)

In [ ]:
training_args = TrainingArguments(
    output_dir=save_folder+'results',
    eval_strategy='steps',
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
    num_train_epochs=epochs,
    max_steps=max_steps,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_steps = 500,
    eval_steps = 500,
    logging_dir='logs',
    #logging_steps=10,
    save_strategy='steps',
    #save_steps=0.3,
    save_total_limit=2,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    # data_collator=data_collator,
    #compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
import transformers

transformers.logging.set_verbosity_info()

In [ ]:
# Fine-tune the model
start = time.time()

train_result = trainer.train()

end = time.time()

elapsed_time = end - start
minutes, seconds = divmod(elapsed_time, 60)
formatted_time = f"{int(minutes):02}:{seconds:05.2f}"
print(formatted_time)

***** Running training *****
  Num examples = 34,770
  Num Epochs = 9,223,372,036,854,775,807
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 17,385
  Number of trainable parameters = 0
The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples: Unknown
  Batch size = 2
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.


Perplexity: 1.31


In [ ]:
print(trainer.state.log_history)
model_path = '/content/drive/MyDrive/Projects/fine-tuning/models/'+model_name+f'-fine-tuned-{epochs}epoch-{manipulation}-v2-lora'

trainer.save_model(model_path)
model.save_pretrained(model_path)

metrics = train_result.metrics
trainer.save_metrics(split='all', metrics=metrics)
tokenizer.save_pretrained(model_path)

Saving model checkpoint to /content/drive/MyDrive/Projects/fine-tuning/models/gpt2-fine-tuned-1epoch-neutral-R-v2-lora
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resi

[{'loss': 4.6657, 'grad_norm': 0.8146650195121765, 'learning_rate': 1.827437446074202e-05, 'epoch': 0.08628127696289906, 'step': 500}, {'eval_loss': 0.8934206962585449, 'eval_runtime': 1.8852, 'eval_samples_per_second': 26.522, 'eval_steps_per_second': 13.261, 'epoch': 0.08628127696289906, 'step': 500}, {'loss': 0.4589, 'grad_norm': 1.3405303955078125, 'learning_rate': 1.654874892148404e-05, 'epoch': 0.1725625539257981, 'step': 1000}, {'eval_loss': 0.47755855321884155, 'eval_runtime': 1.8813, 'eval_samples_per_second': 26.577, 'eval_steps_per_second': 13.289, 'epoch': 0.1725625539257981, 'step': 1000}, {'loss': 0.3611, 'grad_norm': 0.6773692965507507, 'learning_rate': 1.4823123382226059e-05, 'epoch': 0.25884383088869717, 'step': 1500}, {'eval_loss': 0.38372889161109924, 'eval_runtime': 1.8719, 'eval_samples_per_second': 26.71, 'eval_steps_per_second': 13.355, 'epoch': 0.25884383088869717, 'step': 1500}, {'loss': 0.3382, 'grad_norm': 0.1546013206243515, 'learning_rate': 1.30974978429680

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_vers

('/content/drive/MyDrive/Projects/fine-tuning/models/gpt2-fine-tuned-1epoch-neutral-R-v2-lora/tokenizer_config.json',
 '/content/drive/MyDrive/Projects/fine-tuning/models/gpt2-fine-tuned-1epoch-neutral-R-v2-lora/special_tokens_map.json',
 '/content/drive/MyDrive/Projects/fine-tuning/models/gpt2-fine-tuned-1epoch-neutral-R-v2-lora/vocab.json',
 '/content/drive/MyDrive/Projects/fine-tuning/models/gpt2-fine-tuned-1epoch-neutral-R-v2-lora/merges.txt',
 '/content/drive/MyDrive/Projects/fine-tuning/models/gpt2-fine-tuned-1epoch-neutral-R-v2-lora/added_tokens.json',
 '/content/drive/MyDrive/Projects/fine-tuning/models/gpt2-fine-tuned-1epoch-neutral-R-v2-lora/tokenizer.json')

## Do inference

In [ ]:
from transformers import pipeline

In [ ]:
#config = AutoConfig.from_pretrained('gpt2')
# tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForCausalLM.from_pretrained(model_path)

prompt = "Women are "

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
print(generator(prompt))

[{'generated_text': 'Women are \xa0really pissed off at Disney for their\xa0\xa0relationships with\xa0my girlfriend, who is a\xa0f**kin \xa0of a mother, who had\xa0meh from\xa0the\xa0 first\xa0day\xa0 of\xa0her life'}]


In [ ]:
generator('men are')

[{'generated_text': 'men are said to have suffered great stress as the day passed. It was believed that a young boy, whom police had met in the street who was said to be the leader of Arakant Muslims, died. Although no information was given as to'}]

In [ ]:
generator('non-binary people are')

[{'generated_text': 'non-binary people are not allowed on school property.'}]

In [ ]:
generator('the people who work on a ship are called ')

[{'generated_text': "the people who work on a ship are called \xa0loyal\xa0to their country and do whatever they please, they are not the\xa0guardians of the land and don't\xa0believe in them because of their\xa0confessionals to "}]